In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt

embory_option = ['HDRL', 'zhuo_wt_BV82', 'zhuo_wt_RW10348', 'zhuo_wt_RW10425', 'zhuo_wt_RW10434', 'zhuo_wt_RW10714']
AI_CELL = 'Cpaaa'
TARGET_CELL = 'ABarpaapp'
RESOLUTION = 0.254
def load_data(embryo, number):
    """
    input:
        embryo: string, choose from five path name in embory_option
        number: int, choose which embryo to load data
    output:
        data_dicts:dict,dict[nuclei stage][name key][pos/radius],containing postion, radius, cell name
        ai_first_appear: int, 'Cpaaa' first appear nuclei stage
        ai_last_appear: int, 'ABarpaapp' first appear nuclei stage
        target_first_appear: int, 'Cpaaa' die nuclei stage
        target_last_appear: int, 'ABarpaapp' die nuclei stage
    """
    pos_a = []
    radius_a =[]
    cell_name_a = []
    data_dicts = []
    ai_list = []
    target_list = []
    nuclei_num = len(os.listdir('./WT50_release_revised/%s/%02d/nuclei' % (embryo,number)))
    if embryo == 'HDRL':
        nuclei_num = 334
    for i in range (nuclei_num - 1):
        path = './WT50_release_revised/%s/%02d/nuclei/t%03d-nuclei' % (embryo,number,i + 1)
        pos = []
        radius = []
        cell_name = []
        with open(path) as file:
            for line in file:
                line = line[:len(line)-1]
                vec = line.split(', ')
                if vec[9] == '':
                    continue
                else:
                    id = int(vec[0])
                    pos.append(np.array(((float(vec[5])*RESOLUTION), (float(vec[6])*RESOLUTION), (float(vec[7])))))
                    radius.append(float(vec[8]) / 2)
                    cell_name.append(vec[9])
        pos_a.append(pos)
        radius_a.append(radius)
        cell_name_a.append(cell_name)
        pos_radius = list(zip(pos, radius))
        data_dict = dict(zip(cell_name, pos_radius))
        data_dicts.append(data_dict)
    
    for i in range(len(cell_name_a)):
        if AI_CELL in cell_name_a[i]:
            ai_list.append(i)
        if TARGET_CELL in cell_name_a[i]:
            target_list.append(i)
    if ai_list != []:
        ai_first_appear = ai_list[0]
        target_first_appear = target_list[0]
        ai_last_appear = ai_list[-1]
        target_last_appear = target_list[-1]
    else:
        ai_first_appear = 0
        target_first_appear = 0
#     np.save('original ai location', pos_a[168:196])
    print(pos_a[0])
    return data_dicts,ai_first_appear,ai_last_appear,target_first_appear,target_last_appear

In [2]:
#######Single Embryo distance figure######
embory_option = ['HDRL', 'zhuo_wt_BV82', 'zhuo_wt_RW10348', 'zhuo_wt_RW10425', 'zhuo_wt_RW10434', 'zhuo_wt_RW10714']

def Embryo_dis(embryo, number, figname = 'Embryo_distance'):
    """
    input:
        embryo:int, choose from option above
        number: int, range:1-10
        figname: file name of the fig wanted to be save
    output:
        Distance_to_target
        Distance to target cell over time figure
    """
    if embryo == 0:
        time_resolution = 1
        shift_point = 25
    else: 
        time_resolution = 1.25
        shift_point = 20
    
    data_dicts,ai_first_appear,ai_last_appear,target_first_appear,target_last_appear = load_data(embory_option[embryo],number)
    ####Starting and ending stage, starting point and the end points (x,y,z) of these two cells
    print('Embryo_name:%s%02d'% (embory_option[embryo], number))
    print('AI cell \nappear stage: (%s,%s)' % (ai_first_appear,ai_last_appear))
    print('starting location {} \nending location   {}'.format(data_dicts[ai_first_appear][AI_CELL][0],data_dicts[ai_last_appear][AI_CELL][0]))
    print('\nTARGET cell \nappear stage: (%s,%s)'% (target_first_appear,target_last_appear))
    print('starting location {} \nending location   {}'.format(data_dicts[target_first_appear][TARGET_CELL][0],data_dicts[target_last_appear][TARGET_CELL][0]))
    ####Maximum distance using starting stage of AI cell
    pos = []
    
    for key in data_dicts[target_first_appear].keys():
        pos.append(data_dicts[target_first_appear][key][0])
    distances = []
    for i in range(len(pos)):
        for j in range(len(pos)):
            distances.append(np.linalg.norm(pos[i]-pos[j])) 
    print("\nMaximum distance: %d"%max(distances))
    
    ###Distance to target
    distance_to_target = []
    begin = max(ai_first_appear,target_first_appear)
#     begin = ai_first_appear + 15
    difference = target_first_appear - ai_first_appear
    end = min(ai_last_appear,target_last_appear)
    print(begin,end,difference)
    for i in range(end-begin):
        distance_to_target.append(np.linalg.norm(data_dicts[begin + i][AI_CELL][0]- \
                                        data_dicts[begin + i][TARGET_CELL][0]))        
    ##shifting range
    distance_to_target_mid = (max(distance_to_target) + min(distance_to_target)) / 2
    distance_to_target_mid_index = np.argmin(np.abs(np.array(distance_to_target)-distance_to_target_mid))
    print("average_y_mid: {}\ny_mid_index: {}\nactual_y_mid[y_mid_index]: {}\n".format(distance_to_target_mid, distance_to_target_mid_index,distance_to_target[distance_to_target_mid_index]))
    shift_num = distance_to_target_mid_index - shift_point
    print("shift step: %f"% shift_num)
    
    ###Draw distance figure
    fig = plt.figure(1)
    plt.title("Distance to target cell over time")
    plt.xlabel("Time(s)")
    plt.ylabel("Distance to target cell(um)")
    plt.plot([time_resolution * k for k in range(len(distance_to_target))],distance_to_target)
    plt.savefig('%s.png' % figname, bbox_inches='tight')
    plt.show()
    return distance_to_target, shift_num, distance_to_target_mid_index, difference


In [3]:
###Fig2D & HDRL embryo
"""
ZD_BV82_WT_20110419_3_s2_emb3_edited --> (1,5)
ZD_BV82_WT_20110426_1_s1_emb3_edited --> (1,7)
ZD_RW10348_WT_20110126_2_s2_emb2_edited --> (2,4)
ZD_RW10348_WT_20110427_3_s1_emb1_edited --> (2,5)
ZD_RW10348_WT_20110427_3_s1_emb2_edited --> (2,6)
ZD_RW10348_WT_20110427_3_s4_emb2_edited --> (2,10)
ZD_RW10425_WT_20110428_3_s2_emb1_edited --> (3,8) #temporarily removed
ZD_RW10434_20110126_2_s4_emb3_edited --> (4,5)
ZD_RW10434_WT_20110502_1_s3_emb1_edited --> (4,9) #Removed
ZD_RW10714_WT_20110724_1_s2_emb1_edited --> (5,4)
ZD_RW10714_WT_20110724_1_s3_emb1_edited --> (5,7)
"""
fig_2D_list = [(1,5), (1,7), (2,4), (2,5), (2,6), \
               (2,10), (4,5), (5,4), (5,7)]
HDRL_embryo = (0,1)

time_resolution_2D = 1.25
time_resolution_HDRL = 1
distances = []
shift_nums = []
distance_to_target_mid_indexs = []
differences = []
for i in range(len(fig_2D_list)):
    distance, shift_num, distance_to_target_mid_index, difference = Embryo_dis(fig_2D_list[i][0],fig_2D_list[i][1],figname = 'Embryo_fig2D%02d'% i)
    distances.append(distance)
    shift_nums.append(shift_num)
    distance_to_target_mid_indexs.append(distance_to_target_mid_index)
    differences.append(difference)
distance_HDRL,shift_num_HDRL,distance_to_target_mid_index_HDRL, difference_HDRL = Embryo_dis(HDRL_embryo[0],HDRL_embryo[1],figname = 'Embryo_HDRL')


FileNotFoundError: [Errno 2] No such file or directory: './WT50_release_revised/zhuo_wt_BV82/05/nuclei'

In [4]:
#Plot
fig = plt.figure(figsize=(15,15))
# for i in range(len(distances)):
for i in [0,1,2,3,4,7]:
    plt.plot([time_resolution_2D * k for k in range(len(distances[i]))],distances[i], linewidth = 5)
for i in [0,1,2,3,4,7]:
    plt.scatter(distance_to_target_mid_indexs[i] * time_resolution_2D,distances[i][distance_to_target_mid_indexs[i]], c = 'r',s = 600)
#     plt.annotate("MidPoint",(distance_to_target_mid_indexs[i] * time_resolution_2D,distances[i][distance_to_target_mid_indexs[i]]),fontsize = 40)
# plt.plot([time_resolution_HDRL * k for k in range(len(distance_HDRL))],distance_HDRL, label = "HDRL")
# plt.annotate("Mid Point-HDRL",(distance_to_target_mid_index_HDRL * time_resolution_HDRL,distance_HDRL[distance_to_target_mid_index_HDRL]))

plt.scatter(distance_to_target_mid_indexs[0] * time_resolution_2D,distances[0][distance_to_target_mid_indexs[0]], c = 'r',s = 400,label = 'Midpoint')


# plt.title("Distance to target cell over time")
plt.xlabel("Time (minutes)", fontsize = 45)
plt.ylabel("Distance to Target (um)",fontsize = 45)
# plt.savefig('Embryo_distance_Fig2D.png', bbox_inches='tight')
# plt.legend(loc="center right")
plt.xticks(ticks = np.arange(0,45,2),labels = np.arange(-15,30,2),fontsize=24)
plt.yticks(ticks = np.arange(0,24),labels = np.arange(0/0.254,24/0.254,1/0.254).astype(int),fontsize=24)
plt.yticks(fontsize=24)
plt.xlim(xmin =15, xmax = 38)
plt.ylim(ymin = 8, ymax = 24)
plt.legend(fontsize = 45)
plt.show()

IndexError: list index out of range

<Figure size 1080x1080 with 0 Axes>